# Анализ базы данных сервиса для чтения книг 

## Описание проекта
Компания купила крупный стартап-сервис для чтения книг по подписке. Руководство компании поставило задачу проанализировать базу данных приобретенного сервиса. 
Известно, что база содержит информациию о книгах, издательствах, авторах, а также пользовательских обзорах книг. Подробное изучение этих данных поможет сформулировать ценностное предложение для нового продукта.

## Задачи исследования:
1. Посчитать сколько книг вышло после 1 января 2000 года,
2. Для каждой книги постичать количество обзоров и средню оценку,
3. Определить издательство, которое выпустило наибольшее количество книг толще 50 страниц — так вы исключите из анализа брошюры.
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Основная часть

### Импорт библиотек и подключение базы данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных

In [3]:
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Изучение таблицы book

In [5]:
# Выводим 5 строк таблицы book
query = '''SELECT * FROM books LIMIT 5'''

In [6]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
#считаем количество строк таблицы book
query = '''
SELECT COUNT(book_id)
FROM books AS b'''

In [8]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,1000


### Изучение таблицы authors

In [9]:
# Выводим 5 строк таблицы
query = '''
SELECT *
FROM authors AS au
LIMIT 5 '''

In [10]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [11]:
#считаем количество строк таблицы
query = '''
SELECT COUNT(author_id)
FROM authors AS au '''

In [12]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,636


### Изучение таблицы publishers

In [13]:
# Выводим 5 строк таблицы
query = '''
SELECT *
FROM publishers AS pub
LIMIT 5 '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [14]:
# считаем количество строк
query = '''
SELECT COUNT(publisher_id)
FROM publishers AS pub '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,340


### Изучение таблицы ratings

In [15]:
# Выводим 5 строк таблицы
query = '''
SELECT *
FROM ratings AS r
LIMIT 5 '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [16]:
# считаем количество строк
query = '''
SELECT COUNT(rating_id)
FROM ratings AS r '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,6456


### Изучение таблицы reviews

In [17]:
# Выводим 5 строк таблицы
query = '''
SELECT *
FROM reviews AS rew
LIMIT 5 '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
# считаем количество строк
query = '''
SELECT COUNT(review_id)
FROM reviews AS rew '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


### Задание 1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [19]:
query = '''
SELECT COUNT(book_id)
FROM books AS b
WHERE publication_date > '2000-01-01'
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Ответ: После 1 января 2000 года вышло 819 книг.

### Задание 2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [20]:
query = '''
SELECT DISTINCT b.book_id,
       b.title,
       COUNT(DISTINCT r.review_id) AS count_review,
       AVG(rg.rating) AS avg_rating
FROM books AS b 
LEFT OUTER JOIN reviews AS r ON b.book_id=r.book_id 
LEFT OUTER JOIN ratings AS rg ON r.book_id=rg.book_id
GROUP BY DISTINCT b.book_id, b.title
ORDER BY COUNT(DISTINCT r.review_id) DESC
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,207,Eat Pray Love,6,3.395833
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,497,Outlander (Outlander #1),6,4.125000
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,NaN
996,221,Essential Tales and Poems,0,NaN
997,387,Leonardo's Notebooks,0,NaN
998,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,NaN


Ответ: Для каждой книги посчитано количество обзоров и средняя оценка. Есть книги, на которые нет ни одного обзора и оценки. 

### Задание 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [21]:
query = '''
SELECT p.publisher_id,
       p.publisher,
    COUNT(b.book_id) AS count_book
FROM publishers AS p 
LEFT JOIN books AS b ON p.publisher_id = b.publisher_id 
WHERE num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count_book
0,212,Penguin Books,42


Ответ: Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books под id-номером 212 (42 книги).

### Задание 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [22]:
query = '''
SELECT a.author
FROM authors a 
WHERE a.author_id IN (SELECT b.author_id
                      FROM books b
                      WHERE b.book_id IN (SELECT book_id_new
                                          FROM (SELECT book_id AS book_id_new,
                                                       AVG(r2.rating) AS avg_rating
                                                 FROM (SELECT count(r.rating_id) AS count_rating,
                                                              b.book_id AS bk
                                                       FROM ratings AS r 
                                                       LEFT JOIN books AS b ON r.book_id = b.book_id
                                                       GROUP BY b.book_id
                                                       HAVING count(r.rating_id) >= 50) AS t1
                                                 LEFT JOIN ratings r2 ON t1.bk = r2.book_id
                                                 GROUP BY book_id
                                                 ORDER BY AVG(r2.rating) DESC
                                                 LIMIT 1) AS only_book))
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author
0,J.K. Rowling/Mary GrandPré


Ответ: J.K. Rowling/Mary GrandPré — автор с самой высокой средней оценкой книг, если учитывать книги, которые имеют 50 и более оценок.

### Задание 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [23]:
query = '''
SELECT AVG(count_reviews)
FROM (SELECT rev.username,
             COUNT(rev.review_id) AS count_reviews
      FROM reviews AS rev
      WHERE rev.username IN (SELECT t1.username AS username
                             FROM (SELECT r.username,
                                           COUNT(r.rating_id) AS count_comment
                                   FROM ratings AS r
                                   GROUP BY r.username
                                   HAVING COUNT(r.rating_id) > 48) AS t1)
      GROUP BY rev.username) AS final_t
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Ответ: Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24.

## Выводы по проекту

###### Количество строк в таблицах:
* book - 1000,
* authors - 636,
* publishers - 340,
* ratings - 6456,
* reviews - 2793.

###### Задания
1. После 1 января 2000 года вышло 819 книг.
2. Для каждой книги посчитано количество обзоров и средняя оценка. Есть книги, на которые нет ни одного обзора и оценки.
3. Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books под id-номером 212 (42 книги).
4. J.K. Rowling/Mary GrandPré — автор с самой высокой средней оценкой книг, если учитывать книги, которые имеют 50 и более оценок.
5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24.